In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [5]:
def keywords(id):
    word=''
    key_url='https://www.imdb.com/title/tt'+id+'/keywords/'
    #https://www.imdb.com/title/tt'+id_+'/'
    response_keyword=requests.get(key_url,headers=headers)
    content_keyword=response_keyword.content
    soup_keyword=BeautifulSoup(content_keyword,'html.parser')
    keyword_elements=soup_keyword.find_all('a',class_='ipc-metadata-list-summary-item__t')
    for element in keyword_elements:
     word += ', ' + element.text
    return word[2:len(word)]
   

In [6]:
def release_info(id):
    key_url='https://www.imdb.com/title/tt'+id+'/releaseinfo/'
    #https://www.imdb.com/title/tt'+id_+'/'
    response_release_info=requests.get(key_url,headers=headers)
    content_release_info=response_release_info.content
    soup_release_info=BeautifulSoup(content_release_info,'html.parser')
    date=soup_release_info.find('span',class_='ipc-metadata-list-item__list-content-item')
    try:
     if soup_release_info.find('article',class_='sc-8eb80a87-0 hQQXzj').text is not None: 
         return 'NaN'

    except:
       return date.text    
       
    


In [7]:
def runtime(id):
    key_url='https://www.imdb.com/title/tt'+id+'/technical/'
    #https://www.imdb.com/title/tt'+id_+'/'
    response_runtime=requests.get(key_url,headers=headers)
    content_runtime=response_runtime.content
    soup_runtime=BeautifulSoup(content_runtime,'html.parser')
    runtime=soup_runtime.find_all('span',class_='ipc-metadata-list-item__list-content-item--subText')
    color=soup_runtime.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    try:
     return runtime[0].text,color[1].text
    except:
        return 'NaN',color[1].text
           
 

In [8]:
runtime('1517268')

('(114 min)', 'Color')

In [9]:
df={'Id':[],'Title':[],'Storyline':[],'Director':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}

In [10]:
%%time
i=0
for q in range(0,10):
    for w in range(0,10):
        for e in range(0,10):
            for r in range(0,10):
                for t in range(0,10):
                    for y in range(0,10):
                        for u in range(0,10):
                          try:  

                            id_=str(q)+str(w)+str(e)+str(r)+str(t)+str(y)+str(u)
                            w_url='https://www.imdb.com/title/tt'+id_+'/'
                            response=requests.get(w_url,headers=headers)
                            content=response.content
                            soup=BeautifulSoup(content,'html.parser')
                            Title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            try:
                             storyline=soup.find_all('div',class_='ipc-html-content-inner-div')[2].text
                            except:
                                storyline='NaN'

                            Director=soup.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')[0].text
                            Date=release_info(id_)
                            Runtime,colr=runtime(id_)
                            Rating=(soup.find_all('span',class_='sc-bde20123-1 iZlgcd')[0]).text
                            country=soup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base')
                            df['Country'].append(country)
                            key_words=keywords(str(id_))
                            i=i+1
                            print(i,w_url)
                            df['Id'].append(id_)
                            df['Title'].append(Title)
                            df['Storyline'].append(storyline)
                            df['Director'].append(Director)
                            df['Date'].append(Date)
                            df['Runtime'].append(Runtime)
                            df['Color'].append(colr)
                            df['Rating'].append(Rating)
                            df['keywords'].append(key_words)
                          except:
                            pass
                            
                
            break
        break
    break


                        

1 https://www.imdb.com/title/tt0000001/
2 https://www.imdb.com/title/tt0000002/
3 https://www.imdb.com/title/tt0000003/
4 https://www.imdb.com/title/tt0000004/
5 https://www.imdb.com/title/tt0000005/
6 https://www.imdb.com/title/tt0000006/
7 https://www.imdb.com/title/tt0000007/
8 https://www.imdb.com/title/tt0000008/
9 https://www.imdb.com/title/tt0000009/
10 https://www.imdb.com/title/tt0000010/
11 https://www.imdb.com/title/tt0000011/
12 https://www.imdb.com/title/tt0000012/
13 https://www.imdb.com/title/tt0000013/
14 https://www.imdb.com/title/tt0000014/
15 https://www.imdb.com/title/tt0000015/
16 https://www.imdb.com/title/tt0000016/
17 https://www.imdb.com/title/tt0000017/
18 https://www.imdb.com/title/tt0000018/
19 https://www.imdb.com/title/tt0000019/
20 https://www.imdb.com/title/tt0000020/
21 https://www.imdb.com/title/tt0000021/
22 https://www.imdb.com/title/tt0000022/
23 https://www.imdb.com/title/tt0000023/
24 https://www.imdb.com/title/tt0000024/
25 https://www.imdb.com/t

In [ ]:
i

In [ ]:
main_df=pd.DataFrame(df)

NameError: name 'pd' is not defined